# Capstone Project - The Battle of Neighborhoods

# Brooklyn Food Truck Location Recommendations

# I. PURPOSE

This document provides the details of my peer reviewed assignment for the IBM Data Science Professional Certificate Capstone project.

# II. INTRODUCTION

New York City is the largest city in U.S.A by population, which also encompasses five divisions called boroughs.  Of the five boroughs we will focus on Brooklyn, which is known for its night life and great food.

# III. Objective

In this project, we will studies the areas of Brooklyn in order to determine which neighborhoods have the highest concentration of bars in order to solve our business problem.  

## A discription of the problem and discussion of the background. (15 marks)

Our business problem is that we are working with a Food Truck business owner who is trying to determine where he should take his food truck for the greatest chance of success.<br/>
The aim of this project is to segment area's of Brooklyn's neighborhoods based on the concentration of bars using the Foursquare data.  <br/>We therorize that the best places to take a food truck at night is near areas that have high concentrations of bars.<br/>  This theory is based on two main principals.<br/>
1. Not all bars have a kitchen and sell food, so targeting these areas fills a need for our customers.<br/>
2. People going home from the barsare more likely to buy food to enjoy late at night when many other food options are closed.<br/>

Thus the area's with the highest concentration of bars will have the highest concentrations of potential customers.<br/>




## A description of the data and how it will be used to solve the problem. (15 marks)

We will be using the Foursquare Places API for our data source. https://developer.foursquare.com/docs/api<br/>
From this API we will use the Venue Categories and Venue Location data to map our data on a Folium map.<br/>
From here we will use K-Means clustering to find the ideal locations to send the Food Trucks to.